# Creación de csv con las provincias y municipios

Al principio del proyecto nos dimos cuenta de que necesitábamos un CSV con las regiones para agrupar las subvenciones de la BDNS, lo que permitiría conocer como están distribuidas.

Por ende, cogimos los datos para montar el CSV de 4 sitios web:

-codccaa - https://www.ine.es/daco/daco42/clasificaciones/codccaa.xls

-codprov - https://www.ine.es/daco/daco42/clasificaciones/codprov.xls

-codmun - https://www.ine.es/daco/daco42/codmun/codmun11/11codmun.xls

-comarcas99_metodologia - https://www.ine.es/daco/daco42/agricultura/comarcas99_metodologia.xls

In [1]:
import pandas as pd
import numpy as np

Importamos los CSVs:

In [2]:
ccaa = pd.read_csv('codccaa.tsv',
                   header = 1, sep = '\t').iloc[:,:2]

provincias = pd.read_csv('codprov.tsv',
                         header = 1, sep = '\t').iloc[:,:2]

municipios = pd.read_csv('codmun.tsv',
                         header = 1, sep = '\t')

comarcas = pd.read_csv('comarcas99_metodologia.tsv',
                       sep = '\t').iloc[:,:3]                         

FileNotFoundError: [Errno 2] No such file or directory: 'codccaa.tsv'

Creamos una función para eliminar comas, reordenar y juntar los nombres compuestos.

In [ ]:
def reorder_names(datos):
    for i in range(len(datos)):
        for n in datos:
            if isinstance(datos[n][i],np.integer):
                continue
            if ',' in datos[n][i]:
                datos[n][i] = datos[n][i].split(',')[1][1:] + ' ' + datos[n][i].split(',')[0]

reorder_names(ccaa)
reorder_names(provincias)
reorder_names(municipios)

El siguiente paso ahora es juntar los diferentes datos y renombrar las columnas.

In [ ]:
listado = pd.merge(pd.merge(municipios,
                            ccaa,
                            left_on = 'CODAUTO',
                            right_on = 'CODIGO',
                            how = 'left'),
                   provincias,
                   left_on = 'CPRO',
                   right_on = 'CODIGO',
                   how = 'left')[['NOMBRE ',
                                  'LITERAL_y',
                                  'LITERAL_x',
                                  'CMUN',
                                  'CPRO']]

# Rename columns
listado.rename(columns = {'LITERAL_x':'ccaa',
                          'LITERAL_y':'provincia',
                          'NOMBRE ':'municipio',
                          'CMUN':'codmun',
                          'CPRO':'codpro'},
               inplace = True)

comarcas.rename(columns = {'FUENTE: Ministerio de Agricultura, Pesca y Alimentación':'provincia',
                           'Unnamed: 1':'comarca',
                           'Unnamed: 2':'municipio'},
                inplace = True)

Aquí encontramos un problema, los nombres de comarca y los IDs de los municipios estan en la misma columna por lo que tuvimos que separarlos.

### 1. Dividir comarcas  y municipios IDs

In [ ]:
comarcas['codmun'] = ''

for i in range(len(comarcas.comarca)):
    if isinstance(comarcas['comarca'][i],str):
        if comarcas.comarca[i][0] != 'C':
            comarcas.codmun[i] = comarcas.comarca[i]
            comarcas.comarca[i] = np.nan
        else:
            comarcas.comarca[i] = comarcas.comarca[i].split(':')[-1][1:]

### 2. Sustituir NaNs

In [ ]:
comarcas['provincia'].fillna(method = 'ffill',
                             inplace = True)
comarcas['comarca'].fillna(method = 'ffill',
                             inplace = True)
comarcas = comarcas.dropna().reset_index(drop = True)                             


### 3. Nombres compuestos

In [ ]:
for i in range(len(comarcas.municipio)):
    if '(' in str(comarcas.municipio[i]):
        comarcas.municipio[i] = comarcas.municipio[i].split('(')[1][:-1] + ' ' + comarcas.municipio[i].split('(')[0][:-1]

### 4. Unificar IDs de municipios y tipos de variables

In [ ]:
for i in comarcas.index:
   while comarcas.codmun[i][0] == '0':
       comarcas.codmun[i] = comarcas.codmun[i][1:]


comarcas.codmun = pd.to_numeric(comarcas.codmun)


### 5. Crear un DataFrame con IDs y nombres de provincias

In [ ]:
indices = pd.DataFrame(comarcas.provincia.unique(),
                       columns = ['provincia'])
indices['codpro'] = ''

for i in indices.index:
    indices.codpro[i] = i + 1

indices.codpro = pd.to_numeric(indices.codpro)

#### 6. Unir todos los datasets en uno solo

In [ ]:

regiones = pd.merge(listado,
                    pd.merge(comarcas,
                             indices,
                             on = 'provincia',
                             how = 'left')[['comarca',
                                            'codmun',
                                            'codpro']],
                    right_on = ['codmun',
                                'codpro'],
                    left_on = ['codmun',
                               'codpro'],
                    how = 'left').drop(columns = ['codpro',
                                                  'codmun'])[['ccaa',
                                                              'provincia',
                                                              'comarca',
                                                              'municipio']]




### 7. Guardar los datos en un CSV

In [ ]:
regiones.to_csv('/Users/enriquecarnerofernandez/Documents/BDNS/cured_data/regiones.csv',
                index = False)